## Peptidergic Network Autoencoder Evaluation
Authors: Leila Elabbady, Rohan Gala, Uygar Sumbul
Allen Institute of Brain Science
2018-2019

Raw hierarchical tree file: /data/rnaseqanalysis/shiny/facs_seq/mouse_V1_ALM_20180520/dend.RData
tree_20180520.csv was obtained using `dend_functions.R` and `dend_parents.R` functions (Ref. Rohan/Zizhen)

To visually inspect the original tree, visit:
http://molgen-shiny.corp.alleninstitute.org/heatmap_2017/?db=/data/rnaseqanalysis/shiny/facs_seq/mouse_V1_ALM_20180520

In [ ]:
import matplotlib
matplotlib.use('Agg')
from scipy.stats import multivariate_normal as mvn
import numpy as np
import scipy.io as sio
import feather
import pandas as pd
import datetime
import os
import csv
import matplotlib.pyplot as plt

Merge( ) will execute one merge by grabbing the lowest parent node in the hierarchical tree and setting the child node values to its own. 

In [ ]:
def merge(y,child,parent,clusterID,lookup,merged_nodes):
    minind = np.nanargmin(y)
    this_parent = child[minind]
    c_ind = np.where(parent==this_parent)[0]
    lookup[this_parent]=max(lookup.values())+1
    for i in c_ind:
        l = child[i]
        if lookup.has_key(l):
            old_clusterID = lookup[l]
            old_ids = np.where(clusterID==old_clusterID)[0]  
#             child[i]=this_parent
#             y[minind]=np.nan 
            for oi in old_ids:
                clusterID[oi]=lookup[this_parent]
        else:
            #removing glia from dictionary and setting height to none
            lookup.pop(this_parent, None)
            
        child[i]=this_parent
        y[minind]=np.nan
    #keeping track of merged nodes
    merged_nodes.append(this_parent)
    return merged_nodes




Calc_rindex( ) will return the average raw resolution index based on the height of the lowest correct mapping on the tree for each cell of the test set.

In [ ]:
def calc_rindex(predict_history,heights,children,neuron_h):
    res_indices =np.ones(predict_history.shape[0])
    for cell_i in range(predict_history.shape[0]):
        correct = np.where(predict_history[cell_i,:]+1==clusterID_history[cell_i,:])[0]
        lowest = np.min(correct)
        for l in lookup.keys():
            if lookup[l]== clusterID_history[cell_i,lowest]:
                #print('label found: ',l)
                ind = np.where(children == l)
        res_indices[cell_i] = heights[ind]
    res = np.mean(np.array([1-i for i in res_indices]))
    return res,res_indices

Data files. Below, specify the paths to the following files:
* Hierarchical tree csv
* Ground truth mat file
* Autoencoder latent space mat file

In [ ]:
tree_file = '/nas5/peptides/tree_20180520.csv'
ground_truth_file = '/nas5/peptides/mouse_V1_ALM_20180520_6_5byExpression_and_NP18andGPCR29.mat'
latent_space_file = "/nas5/peptides/dualAE_inputZ1_6_5byExpression_and_NP18GPCR29_dim5_run0_iter10K_loss1_100_0.0Dropout_intermediate_50_bat794_neuronsOnly.mat"

Loading  and copying hierarchical information from the tree csv that will later be used for iterative merging and resolution index calcualation

In [ ]:
htree = pd.read_csv(tree_file)
htree = htree[['x','y','leaf','label','parent','col']]
htree['leaf'] = htree['leaf'].values==True
htree = htree.sort_values(by=['y','x'], axis=0, ascending=[True,True]).copy(deep=True)
htree = htree.reset_index(drop=True).copy(deep=True)

#Copy of fields used for merging:
y = htree['y'].values.copy()
y[htree['leaf'].values]=np.nan
child = htree['label'].values.copy()
parent = htree['parent'].values.copy()
leaf = htree['leaf'].values.copy()

#for res_score
children = htree['label'].values.copy()
heights = htree['y'].values.copy() #for res_score


To update cluster IDs as the tree is merged, we set up a look-up dictionary mapping the cluster/ID pairings from the ground truth file. As new clusters are formed after each merge, a new cluster ID will be added to the dictionary.

In [ ]:
#Loading clusters and IDs from ground truth file
gt_contents = sio.loadmat(ground_truth_file)
clusterID = np.squeeze(np.concatenate(np.concatenate(gt_contents["clusterID"])))
clusters = np.squeeze(np.concatenate(np.concatenate(gt_contents["cluster"])))

lookup = {}
for c, id in zip(clusters,clusterID):
    lookup[str(c)]= id

Setting up the train and test sets for the GMM. In this case, the test set is the first 8% of the dataset, consistent with the test/train split used in the autoencoder training. Within the code, the training and test sets are referred to as e2 and et2 respectively.

In order to track the performance of the GMM at every cell in the test set at each cut of the tree, two 'history' matrices are stored. One that records the predicted cluster assignment(predict_history) and a second to record the true cluster assignment (clusterID_history

In [ ]:
#Loading latent space contents
mat_contents = sio.loadmat(latent_space_file)

e2 = np.array(mat_contents["e2"])
et2 = np.array(mat_contents["et2"])
thisRun = 0
foldCount = 13
foldSize = gt_contents["logOnePlusGeneExpression"].shape[0] / foldCount
all_ind = np.arange(gt_contents["logOnePlusGeneExpression"].shape[0])
val_ind = np.arange(foldSize+1)
train_ind  = np.asarray(list(set(all_ind) - set(val_ind)))
cuts = np.where(leaf ==False )[0]
predict_history = np.zeros((val_ind.shape[0],cuts.shape[0]))
clusterID_history = np.zeros((val_ind.shape[0],cuts.shape[0]))


For each cut of the hierarchical tree, this cell will train a GMM and assign new ClusterIDs to the given test set. Once those values have been added to the predict_history matrix, the tree will be merged at it's lowest point, thereby creating a new cut, and so forth until there is only one root node left. 

In [ ]:
cut=0
merged_nodes = [] 

while cut < cuts.shape[0]:
    cluster_num = int(max(clusterID))
    clusterLik = np.zeros((mat_contents["et2"].shape[0], cluster_num))
    clusterMeans = np.empty((cluster_num,int(e)))
    clusterCovs = [None]*(cluster_num) 

    for cluster in range(cluster_num):
        clusterSamples = e2[clusterID[train_ind]==cluster+1,:]
        if clusterSamples.shape[0]>0:
            means = np.mean(clusterSamples, axis =0)
            clusterMeans[int(cluster),:] = means
            if clusterSamples.shape[0]<50:
                clusterCovs[int(cluster)] = np.diagonal(np.cov(clusterSamples,rowvar=False))
            else:
                clusterCovs[int(cluster)] = np.cov(clusterSamples,rowvar=False)

            clusterLik[:, int(cluster)] = mvn.pdf(et2,clusterMeans[int(cluster),:], 
                                                         clusterCovs[int(cluster)])
            
    valClusterID = clusterID[:val_ind.shape[0]]
    clusterID_history[:,cut] = valClusterID
    clusterAssign = clusterLik.argmax(1)
    predict_history[:,cut] = clusterAssign
    merged_nodes = merge(y,child,parent,clusterID,lookup,merged_nodes)
    cut +=1

Now that we have the prediction and ground truth history of each cell at each cut of the hierarchical tree. We can now quantify the most complex representation of the tree that was mapped correctly by taking the resolution index for each cell at the lowest correctly assigned cluster on the tree.

To account for the fact that our dataset only includes neural cells, the average resolution index is normalized of the tree height of the first neural node, here classified as 'n3'. 

In [ ]:
#grabbing height of n4, first neural node
neuron = np.where(children == 'n3')
neuron_h = heights[neuron]
norm = 1 - neuron_h

#calculating avg resIndex 
res, res_indices = calc_rindex(predict_history,heights,children,neuron_h)
res_norm = float((res-norm)/(1-norm)) #normalizing over n3 (1-0.46)
print(res_norm, res)

In [ ]:
    #Plotting ResIndex per cell-type distribution
    # l = clusters
    # x = clusterID_history[:,0].astype(int)
    # y = 1-res_indices
    # y_norm = ((1-res_indices)-norm)/(1-norm)
    # df = pd.DataFrame({'ClusterID':x, 'ResIndex':y_norm, 'Cell_Types':l})
    # means = df.groupby('Cell_Types').mean()
    #means.to_csv("Cell_resIndex_by_cellType.csv")
    #df.to_csv("resIndex_per_cell.csv")
    # r = means['ResIndex']

plt.bar(y)
y = [0.987,0.91954,0.464032,0.7681964]
std = [0,0,0.039069179374101926,0.0688674766607247]


    # b = means.plot(kind= 'bar',y = mask1, legend = False, stacked = False, color = "lightblue" ,
    #             figsize=(35,10),rot=90,title="Avg ResIndex across cell-type clusters")
    # b = means.plot(kind= 'bar',y = mask2, legend = False, stacked = False, color = "b",
    #             figsize=(35,10),rot=90,title="Avg ResIndex across cell-type clusters")
    # bar = b.get_figure()
    # bar.tight_layout() 
    #bar.savefig('./figures/np_test1.tiff')